In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import pome
from pome import datasource, metrics, signal
import pome.simulator.amount_based as simulator
import datetime

In [2]:
signal_url = 'file:///Users/chen-weihung/Documents/GitRepositories/Alpaca/pome/data/sample_signal.csv'
data_url = 'file:///Users/chen-weihung/Documents/GitRepositories/Alpaca/pome/data/sample_datasource_topix.csv'
sig = signal.load_signal('BBG-USDJPY_Curncy', signal_url, datetime.datetime(2015,1,5), datetime.datetime(2017,1,5))

In [3]:
#print(sig.symbol)
sig.head()

,label,pred
2015-01-05,-1.0,-1.0
2015-01-06,-1.0,-1.0
2015-01-07,1.0,1.0
2015-01-08,-1.0,-1.0
2015-01-09,1.0,1.0


In [4]:
mkt = datasource.load_marketdata(sig.symbol, sig._start_dt, sig._end_dt, data_url)
mkt.head(10)

,lag0_dt_close,lag0_dt_max,lag0_dt_mean,lag0_dt_min,lag0_dt_open,lag0_dt_std,lag10_dt_close,lag10_dt_max,lag10_dt_mean,lag10_dt_min,...,lag8_dt_mean,lag8_dt_min,lag8_dt_open,lag8_dt_std,lag9_dt_close,lag9_dt_max,lag9_dt_mean,lag9_dt_min,lag9_dt_open,lag9_dt_std
2008-01-04,1411.91,1461.31,1421.740312,1403.22,1461.23,12.403812,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000
2008-01-07,1392.71,1408.33,1395.974870,1385.51,1397.37,4.743925,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000
2008-01-08,1403.06,1403.95,1390.597528,1380.61,1387.24,4.971238,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000
2008-01-09,1424.29,1424.30,1405.389444,1379.34,1387.56,13.372210,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000
2008-01-10,1401.36,1422.17,1411.773167,1401.36,1419.76,4.600218,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000
2008-01-11,1377.58,1408.29,1390.157972,1374.63,1407.08,10.122851,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000
2008-01-15,1350.20,1382.59,1362.645333,1345.44,1377.98,11.108080,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000
2008-01-16,1302.37,1333.15,1315.229713,1301.00,1332.90,7.538132,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000
2008-01-17,1330.44,1333.53,1316.566019,1297.40,1311.87,7.887494,0.0,0.0,0.0,0.0,...,1421.740312,1403.22,1461.23,12.403812,0.00,0.00,0.000000,0.00,0.00,0.000000
2008-01-18,1341.50,1344.72,1316.211065,1289.29,1311.58,17.669009,0.0,0.0,0.0,0.0,...,1395.974870,1385.51,1397.37,4.743925,1411.91,1461.31,1421.740312,1403.22,1461.23,12.403812


In [5]:
from pome.labelizer import DiffReturnDiffLabelizer
lbl = pome.generate_labels(
    mkt,
    DiffReturnDiffLabelizer(
        lookahead=3,
        p1_compared_column_name='lag0_dt_max',
        n1_compared_column_name='lag0_dt_min',
        standard_column_name='lag0_dt_close'
    ))
lbl.head()

,label,label_diff
2008-01-04,-1.0,-0.008309
2008-01-07,-1.0,-0.001613
2008-01-08,1.0,0.007423
2008-01-09,-1.0,-0.004708
2008-01-10,1.0,0.009022


In [6]:
from pome.labelizer import ReturnDifferenceLabelizer
lbl = pome.generate_labels(
    mkt,
    ReturnDifferenceLabelizer(
        lookahead=3,
        compared_column_name='lag0_dt_close',
        standard_column_name='lag0_dt_max'
    ))
lbl.head()

,label,label_diff
2008-01-04,1.0,0.034095
2008-01-07,-1.0,-0.003616
2008-01-08,-1.0,-0.021252
2008-01-09,-1.0,-0.022734
2008-01-10,-1.0,-0.008044


In [7]:
from pome.labelizer import FutureDirectionLabelizer
lbl = pome.generate_labels(
    mkt,
    FutureDirectionLabelizer(
        lookahead=3,
        target_column_name='lag0_dt_close'
    ))
lbl.head()


,label,label_diff
2008-01-04,1.0,12.38
2008-01-07,1.0,8.65
2008-01-08,-1.0,-25.48
2008-01-09,-1.0,-74.09
2008-01-10,-1.0,-98.99


In [8]:
metrics.calc_metrics(sig, lbl).to_frame()

,accuracy,diff_w_real_diff,f1_d,f1_u,precision_d,precision_u,pred_count_d,pred_count_u,pred_flip_count,pred_label_diff,pred_ratio_d,pred_ratio_u,real_flip_count,real_label_diff,real_ratio_d,real_ratio_u,recall_d,recall_u
0,0.477051,288.0,0.502646,0.44868,0.422222,0.568773,450.0,269.0,235.0,-181.0,0.625869,0.374131,183.0,107.0,0.425591,0.574409,0.620915,0.37046


In [9]:
trades = simulator.only_take_short(mkt, sig, target_column_name='lag0_dt_close')
trades.head()

,amount,lag0_dt_close,lag0_dt_max,lag0_dt_mean,lag0_dt_min,lag0_dt_open,lag0_dt_std,lag10_dt_close,lag10_dt_max,lag10_dt_mean,...,lag8_dt_mean,lag8_dt_min,lag8_dt_open,lag8_dt_std,lag9_dt_close,lag9_dt_max,lag9_dt_mean,lag9_dt_min,lag9_dt_open,lag9_dt_std
2015-01-05,-1.0,1401.09,1410.26,1399.890956,1388.37,1400.87,6.307549,1353.37,1365.64,1356.776915,...,1383.071351,1374.39,1377.77,3.496153,1352.01,1362.08,1356.642330,1346.37,1347.03,3.034502
2015-01-06,-1.0,1361.14,1377.88,1368.921622,1361.14,1377.53,3.767356,1352.01,1362.08,1356.642330,...,1402.846743,1396.48,1399.92,2.904520,1376.32,1389.69,1383.071351,1374.39,1377.77,3.496153
2015-01-07,0.0,1359.80,1367.61,1362.612442,1353.43,1353.43,2.717900,1376.32,1389.69,1383.071351,...,1410.689368,1407.29,1415.97,2.007968,1409.61,1409.61,1402.846743,1396.48,1399.92,2.904520
2015-01-08,-1.0,1377.67,1383.79,1378.427138,1370.99,1374.91,2.988409,1409.61,1409.61,1402.846743,...,1425.983842,1423.80,1428.27,0.837524,1413.05,1417.27,1410.689368,1407.29,1415.97,2.007968
2015-01-09,0.0,1380.58,1387.24,1380.879165,1375.02,1386.72,2.707087,1413.05,1417.27,1410.689368,...,1421.404454,1418.83,1422.53,1.329430,1426.02,1429.32,1425.983842,1423.80,1428.27,0.837524


In [10]:
metrics.calc_trade_performance(sig, lbl, trades).to_frame().T

,0
accuracy,0.477051
average_loss,-11.363643
average_pl,-0.981804
average_win,13.041832
baseline_pl,402.640000
baseline_pl_percentage,0.287376
diff_w_real_diff,288.000000
f1_d,0.502646
f1_u,0.448680
precision_d,0.422222
